## Equipo ...CriptoBroker...
#### Asignatura: TEL252-S2-2020
#### Alumnos: Christian Fuentes 201530024-0 , Mario Araya 2015 201630003-1 , Sebastián Araneda 201530022-4 , Victor Cortés 201604140-0


### El producto consisten 4 partes fundamentales:
     1.-  Frontend
     2.-  DB
     3.-  Api
     4.-  Sensor

<img src="imgs/CriptoBroker.png">

## 1.- Frontend
En la parte del frontend lo que tenemos es una vista la cual va a mostrar la temperatura de los sensores de forma gráfica, tomando la temperatura de un sensor y gratificando determinado intervalo de tiempo, además del identificador único del sensor. Ésta parte de la arquitectura se comunica por el puerto 3000.

## 2.- Base de datos
La base de datos se encuentra en un servidor alojado en DigitalOcean, es una base datos mysql donde tenemos la tabla "sensores" que contiene las variables id_sensor VARCHAR(128), lectura VARCHAR(512), fecha TIMESTAMP, token VARCHAR(128). Cabe destacar que el único que tiene conexión directa con la base de datos es la API por el puerto 9600. La base de datos guarda el valor de las lecturas encriptadas utilizando el algoritmo AES, con el método de operación OFB.

<img src="imgs/bd.JPG">

## 3.- API
Para nuestro caso de la api rest, esta vez usamos express con con nodejs para recibir las solicitudes de get y post que van a llegar ya sea para pedir la temperatura de la base de datos o para ingresarla cuando el sensor entregue los datos para ser guardados en la base de datos. La API también cumple la función de desencriptar los mensajes.

## 4.- Sensor
Para el caso de nuestro sensor, lo que tenemos es un archivo sensor.py que simula un sensor, mandando la temperatura  encriptada usando el algoritmo de "AES" usando el modo de operación “OFB”, para posteriormente ser enviada en una trama http que contiene un json de la forma 
```
{
         "id_sensor": id_sensor,
         'lectura': data.decode("utf-8")
}
```
donde el "id_sensor" corresponde a un valor que diferencia un sensor de otro en caso de que tengamos más de uno, y la lectura corresponde al valor de temperatura en nuestro caso que nos entrega el sensor.
}
El sensor esta simulado dentro Alphine Linux, la cual es es una distribucion que tiene como objetivo ser ligera y segura por defecto sin dejar de ser útil para tareas de propósito general. Dentro del sistema Aphine Linux se encuentra el programa que simula un sensor, este sensor puede simular diferentes tipos de mediciones para estudio, como lo son temperatura, humedad, ph, emisiones de gases, etc. El código se puede encontrar en el repositorio con el nombre de sensor.py

## Encriptación y seguridad:
Para la encriptación y seguridad del sistema se ha usado el algoritmo simétrico AES, el cual es un algoritmo iterativo que utiliza bloques de 128 bits(16 Bytes) al igual que la llave K y el vector inicial, este último usado para el método de operación OFB implementado con el algoritmo. AES utiliza capas de funciones por cada ciclo en los bloques, sustitución para la confusión, desplazamiento de filas para la difusión, combinación de columnas y adición de llave, donde en el último ciclo no incluye combinación de columnas. Se utilizó el Método de operación OFB ya que es un método probabilístico que produce textos cifrados diferentes, esto permite que no se realicen ataques de análisis de frecuencia. 

A lo largo del funcionamiento del sistema podemos encontrar la encriptación con el método AES OFB en el momento que el sensor envía la información en formato JSON desde un algoritmo albergado dentro del sistema operativo Alpine. Desde ese punto viaja la “Id” del sensor y la información como tal encriptada, junto con el valor del vector inicial para el método OFB, luego al llegar a la API desencriptamos la id del sensor y corroboramos si el valor entregado existe en los registros de sensores existentes, si el valor no coincide, el mensaje es ignorado, por otro lado, si valor coincide registramos la id desencriptada, el mensaje encriptado y el vector inicial. Luego, cada vez que la información sea requerida por un usuario desde el Front-End se extraerá desde la base de datos, se desencriptará el mensaje y se mostrará en pantalla.


## Protocolo de comunicación:

HTTP es el protocolo de comunicación que permite las transferencias de información a través de archivos, en el proyecto usamos este protocolo para poder mandar la temperatura encriptada hacia la api, con esto podemos hacer métodos post a través de python, donde tenemos 2 métodos get y 1 metodo post en la api, de los cuales 1 va a filtrar por la id del sensor y el otro get va a traer todos los datos (get_all), y el post para guardar las lecturas del sensor. Una de las razones principales por la que decidimos usar este protocolo y no usamos MQTT que era uno de los que teníamos en vista al comienzo del proyecto, fue por que todos en el grupo estábamos un poco más familiarizados con el protocolo y el uso del mismo, además como estamos solamente mandando datos en formato de caracteres de bajo costo, el uso del protocolo HTTP no debería ser un problema en nuestro caso. El servidor tiene un firewall que controla el trafico, permitiendo las consultas entrantes solo por el puerto 9600 para trabjar con la api

## Eficiencia

Un cifrado de 128 bits equivale a 2^(128) diferentes combinaciones o claves posibles, lo que corresponde a más de 3,4*10^(38) combinaciones, por lo que frente a un ataque de fuerza bruta es un algoritmo muy eficiente, como es bien sabido AES es un algoritmo simétrico, por lo tanto podemos tener problema de ataque de "man in the middle" por parte de eve, pero esto ya fue resuelto, debido a que tenemos la "clave" pasada de forma "física", por lo tanto no hay posibilidad de tener un problema de "man in the middle" para capturar la llave que desencriptará y encriptará nuestros mensajes. 


## Verificación
Para la verificación es posible ver las capturas que está obteniendo el sensor y comprobar que éstas se encuentren almacenadas con integridad en la base de datos, y además el resultado de desencriptarlas nos entregue el mismo valor que conocemos de la lectura del sensor.

